<!-- README.md is generated from README.Rmd. Please edit that file -->

# hdforce <img src="hdlogo_sm.png" align="right" alt="" width="120" />

**Get your data from the Hawkin Dynamics API**

<!-- badges: start -->
[![Build Status](https://travis-ci.com/laureng-hd/hawkinPy.svg?branch=master)](https://travis-ci.com/laureng-hd/hawkinPy)
[![Coverage Status](https://coveralls.io/repos/github/laureng-hd/hawkinPy/badge.svg?branch=master)](https://coveralls.io/github/laureng-hd/hawkinPy?branch=master)
![Static Badge](https://img.shields.io/badge/hdforce_v0.0.1-beta-orange?style=plastic)
[![Last-changedate](https://img.shields.io/badge/last%20change-2024--04--23-yellowgreen.svg)](/commits/dev_man)
[![license](https://img.shields.io/badge/license-MIT%20+%20file%20LICENSE-lightgrey.svg)](https://choosealicense.com/)
[![Project Status: WIP – Initial development is in progress, but there has not yet been a stable, usable release suitable for the public.](https://www.repostatus.org/badges/latest/wip.svg)](https://www.repostatus.org/#wip)
[![lifecycle](https://img.shields.io/badge/lifecycle-experimental-orange.svg)](https://www.tidyverse.org/lifecycle/#experimental)
[![packageversion](https://img.shields.io/badge/Package%20version-0.0.1-orange.svg?style=flat-square)](commits/dev_man)
[![thanks-md](https://img.shields.io/badge/THANKS-md-ff69b4.svg)](THANKS.md)

<!-- badges: end -->

# How To Use The hdforce Package

hawkinR provides simple functionality with Hawkin Dynamics API. These functions are for use with ‘Hawkin Dynamics Beta API’ version 1.10-beta. You must be an Hawkin Dynamics user with active integration account to utilize functions within the package.

## Functions
This API is designed to get data out of your Hawkin Dynamics server and interact with your data in a more intament way. It is not designed to be accessed from client applications directly. There is a limit on the amount of data that can be returned in a single request (256mb). As your database grows, it will be necessary to use the `from` and `to` parameters to limit the size of the responses. Responses that exceed the memory limit will timeout and fail. It is advised that you design your client to handle this from the beginning. A recommended pattern would be to have two methods of fetching data. A scheduled pull that uses the `from` and `to` parameters to constrain the returned data to only tests that have occurred since the last fetch e.g. every day or every 5 minutes. And then a pull that fetches the entire database since you began testing that is only executed when necessary. A recommended way of doing this is to generate the `from` and `to` parameters for each month since you started and send a request for each either in parallel or sequentially.

This package was meant to help execute requests to the Hawkin Dynamics API with a single line of code. There are 13 functions to help execute 4 primary objectives:

### Dependencies
Like all packages, you will first need to load the neccesary modules. 

In [12]:
# Dependencies
import hdforce
import os
from dotenv import load_dotenv

### Get Access
Use the Refresh Token generated by https://cloud.hawkindynamics.com/integrations to get a valid Access Token. Only the organization administrator account has the ability to generate API tokens. Use this function to initiate access to your data in the cloud. All other hawkinR functions will depend on the values returned from this function.

In hdforce, we can generate and use the access code by creating an object with the `GetAccess` function. This object will be used as an argument in further funcitons.

`GetAccess` requieres an `refreshToken` argument ot be passed, along with an optional `region`. The `refreshToken` is where you will enter your cloud generated unique API token. While this can simply be accomplished by manually inputting a text string, it is good pratice to create an enviornmental variable to store your API key. This can be done a few different ways. Most simply, you can create an .env file and store the token there. This way if you need to create a new token and replace it, you can do so within the file easily. To do this you will need to download and load the dotenv pakcage.

In [ ]:
# Inside your .env file
# 1. API_TOKEN is the variable name, use whichever name you want
# 2. Replace YOUR_API_TOKEN with your actual API API_TOKEN

API_TOKEN=YOUR_API_TOKEN

Now you can use the os and dotenv packages to get your token. 

With the API token stored as an environment variable, we can use it to get our access token for further use in other functions.

In [8]:
# declare .env file as location of environment variables
load_dotenv('.env')

# create object containing token
myToken = os.getenv("API_TOKEN")

# Get Access Token
# region is default to Americas and can be excluded if in that region
token = hdforce.GetAccess(refreshToken= myToken, region= "Americas")

### Hawkin Specific data
While the purpose of the package is to help with accesing data specific to your organization, it may be helpful to store some data that is specific to Hawkin and the Hawkin Dynamics system. Things like test types and test metrics will help you call your data more efficiently and better understand the values you are seeing.

* `GetTypes` - Get the test type names and ids for all the test types in the system. Response will be a data frame containing the tests that are in the HD system.
* `GetMetrics` - Get all the metrics for each test type. Response will be a data frame containing the test cannonical id, test type name, metric id, metric label, metric unit of measure, and description.

### Organization Specific Data
Every organization has data specific to them. And with that, these entities will have unique ids. It is important to have these ids available to make the most of your test calls.
* `GetAthletes()` - Get the athletes for an account. Inactive players will only be included if `inactive` parameter is set to TRUE. Response will be a data frame containing the athletes that match this query.
* `GetTeams()` - Get the team names and ids for all the teams in the org. Response will be a data frame containing the teams that are in the organization.
* `GetGroups()` - Get the group names and ids for all the groups in the org. Response will be a data frame containing the groups that are in the organization.
* `GetTags()` - Get the tag names, ids, and descriptions for tags created by users in your org. Response will be a data frame.

### Get Test Data
This is what you are here for. These functions allow you to call test data in the most efficient and straight forward way. It is encouraged that you take advantage of the `from_`, `to_`, and `sync` parameters. This will help prevent from having calls with large payloads that may timeout and fail.

#### Get Test Arguments
* `from_` =  Optionally supply a time (Unix timestamp) you want the tests from. If you do not supply this
value you will receive every test. This parameter is best suited for bulk exports of historical data.
* `to_` = Optionally supply a time (Unix timestamp) you want the tests to. If you do not supply this value
you will receive every test from the beginning of time or the optionally supplied `from_`
parameter. This parameter is best suited for bulk exports of historical data
* `sync` = The result set will include updated and newly created tests, following the time constraints of `from_` and `to_`. This parameter is best suited to keep your database in sync with the Hawkin database. It cannot and should not be used to fetch your entire database. A recommended strategy would be to have a job that runs on a short interval e.g. every five minutes that sends the `lastSyncTime` that it received as the `from_` parameter with `sync=True`.
* `active` = If True, only active tests are fetched. If False, all tests including inactive ones are fetched. Default is True.

#### Get Test Function
* `GetTests()` - Get the tests for your account. You can specify a time frame `from_`, or `to_`, which the tests should come (or be synced). Response will be a data frame containing the trials within the time range (if specified).
* `GetTestsAth()` - Get only tests of the specified athlete from your organization. You can specify a time frame `from_`, or `to_`, which the tests should come (or be synced). Response will be a data frame containing the trials from the athlete, within the time range (if specified).
* `GetTestsType()` - Get only tests of the specified test type from your organization. You can specify a time frame `from_`, or `to_`, which the tests should come (or be synced). Response will be a data frame containing the trials from that test type, within the time range (if specified).
* `GetTestsTeam()` - Get only tests of the specified teams from your organization. Requieres a `teamId` argument, which expects comma seperated text string (max of 10 teams). You can specify a time frame `from_`, or `to_`, which the tests should come (or be synced). Response will be a data frame containing the trials from those teams, within the time range (if specified).
* `GetTestsGroup()` - Get only tests of the specified groups from your organization. Requieres a `groupId` argument, which expects comma seperated text string (max of 10 groups). You can specify a time frame `from_`, or `to_`, which the tests should come (or be synced). Response will be a data frame containing the trials from those groups, within the time range (if specified).
* `GetForceTime()` - Get the force-time data for a specific test by id. This includes both left, right and combined force data at 1000hz (per millisecond). Calculated velocity, displacement, and power at each time interval will also be included.

## Examples
This is a basic example which shows common workflow:

### Authenticate Session

In [9]:
# Dependencies
import hdforce
import os
from dotenv import load_dotenv
import pandas as pd
from IPython.display import HTML, display
# declare .env file as location of environment variables
load_dotenv('.env')

# create object containing token
myToken = os.getenv("API_TOKEN")

# Get Access Token
# region is default to Americas and can be excluded if in that region
token = hdforce.GetAccess(refreshToken= myToken, region= "Americas")

### Get HD Data

In [12]:
# Test Types
types = hdforce.GetTypes(token=token)

types

id                  name
0  7nNduHeM5zETPjHxvm7s  Countermovement Jump
1  QEG7m7DhYsD6BrcQ8pic            Squat Jump
2  2uS5XD5kXmWgIZ5HhQ3A        Isometric Test
3  gyBETpRXpdr63Ab2E0V8             Drop Jump
4  5pRSUQVSJVnxijpPMck3              Free Run
5  pqgf2TPUOQOQs6r0HQWb           CMJ Rebound
6  r4fhrkPdYlLxYQxEeM78         Multi Rebound
7  ubeWMPN1lJFbuQbAM97s              Weigh In
8  rKgI4y3ItTAzUekTUpvR          Drop Landing

In [13]:
# Test Metrics
metrics = hdforce.GetMetrics(token=token)

metrics[10:14]

canonicalTestTypeId          testTypeName                          id  \
10  7nNduHeM5zETPjHxvm7s  Countermovement Jump            peakBrakingForce   
11  7nNduHeM5zETPjHxvm7s  Countermovement Jump    peakRelativeBrakingForce   
12  7nNduHeM5zETPjHxvm7s  Countermovement Jump          avgPropulsiveForce   
13  7nNduHeM5zETPjHxvm7s  Countermovement Jump  avgRelativePropulsiveForce   

                             label units  \
10              Peak Braking Force     N   
11     Peak Relative Braking Force     %   
12           Avg. Propulsive Force     N   
13  Avg. Relative Propulsive Force     %   

                                          description  
10  The peak instantaneous vertical ground reactio...  
11  The peak instantaneous vertical ground reactio...  
12  The average vertical ground reaction force app...  
13  The average vertical ground reaction force app...

### Organization Data

#### Get Athletes

In [14]:
# Get Athletes
roster = hdforce.GetAthletes(token=token, inactive= False) # inactive is default to Fasle

# Athelte example
roster[roster['name'] =="Lauren Green"]

id          name  \
41  OLbsebtmf81eiwg1AeE5  Lauren Green   

                                                teams                  groups  \
41  [DPMb6ek2mgUNVcg8siSqpnIvE2i2, vW9iEKafhs2Pamf...  [yh8RnOvg56dQNrZGBKWZ]   

    active  external.DPMb6ek2mgUNVcg8siSqpnIvE2i2 external.GradYear  \
41    True                                    NaN              2004   

   external.location  external.uniqueId external.StudentID  
41          Whittier  83keo9wjei939ekd9          SA0042643

#### Get Teams

In [15]:
# Get Teams
teams = hdforce.GetTeams(token=token)

# Team example
teams[0:1]

id                    name
0  DPMb6ek2mgUNVcg8siSqpnIvE2i2  Greenhouse Performance

#### Get Groups

In [16]:
# Get Groups
groups = hdforce.GetGroups(token=token)

# Groups example
groups[0:3]

id     name
0  0Ma1qFABBmGHEONWprec     PLNU
1  6zQBrZ4PJ14JeKDJ68aA  Class24
2  FMlGcJZ4xbH2YTEbZWkk      INF

#### Get Tags

In [17]:
# Get Tags
tags = hdforce.GetTags(token=token)

# Tags example
tags[0:6]

id            name  \
0  9OGdYG2LSyC5xUvm5nVB  Baseball Swing   
1  9z5vLui3IxxBwm8imbo6      SL Balance   
2  ACkShg8G05F1tT8hbmdd             LSS   
3  Afcw45lIkeHFnlUyDeSn            5/10   
4  BV52tk5mf93hgNyOI1te    Loaded Squat   
5  DWbZSQRh7qallnhvpsNB         Push Up   

                                       description  
0                                             None  
1  Single leg balance test. Used with Free Run COP  
2     Long Stretch Shortening rebound cue for CMJR  
3                 5/10 Multi-Rebound test protocol  
4                                             None  
5                                             None

#### Get Tests

In [21]:
# Get Tags
allTests = hdforce.GetTests(token=token)
# Filter my tests
mytests = allTests[allTests['athlete.name'] =="Lauren Green"]
# Slice some rows and columns
df = mytests.iloc[:5,:15]

df

id   timestamp            athlete.id  athlete.name  \
0   9Ytz9g1erMXm3SByTyEd  1689958617  OLbsebtmf81eiwg1AeE5  Lauren Green   
19  G1yfTWvTj3hAveQYz5wH  1690858908  OLbsebtmf81eiwg1AeE5  Lauren Green   
22  YB35oOBAGHNQew0WziDt  1690859091  OLbsebtmf81eiwg1AeE5  Lauren Green   
23  2RnV4tM3J6IW2qYKgqg2  1690859127  OLbsebtmf81eiwg1AeE5  Lauren Green   
26  qNIZaBguZefAyar4oUtu  1690859309  OLbsebtmf81eiwg1AeE5  Lauren Green   

                                        athlete.teams          athlete.groups  \
0   [DPMb6ek2mgUNVcg8siSqpnIvE2i2, vW9iEKafhs2Pamf...  [yh8RnOvg56dQNrZGBKWZ]   
19  [DPMb6ek2mgUNVcg8siSqpnIvE2i2, vW9iEKafhs2Pamf...  [yh8RnOvg56dQNrZGBKWZ]   
22  [DPMb6ek2mgUNVcg8siSqpnIvE2i2, vW9iEKafhs2Pamf...  [yh8RnOvg56dQNrZGBKWZ]   
23  [DPMb6ek2mgUNVcg8siSqpnIvE2i2, vW9iEKafhs2Pamf...  [yh8RnOvg56dQNrZGBKWZ]   
26  [DPMb6ek2mgUNVcg8siSqpnIvE2i2, vW9iEKafhs2Pamf...  [yh8RnOvg56dQNrZGBKWZ]   

    athlete.active athlete.external.GradYear athlete.external.location  \
0             True                      2004                  Whittier   
19            True                      2004                  Whittier   
22            True                      2004                  Whittier   
23            True                      2004                  Whittier   
26            True                      2004                  Whittier   

   athlete.external.uniqueId athlete.external.StudentID  \
0          83keo9wjei939ekd9                  SA0042643   
19         83keo9wjei939ekd9                  SA0042643   
22         83keo9wjei939ekd9                  SA0042643   
23         83keo9wjei939ekd9                  SA0042643   
26         83keo9wjei939ekd9                  SA0042643   

    athlete.external.DPMb6ek2mgUNVcg8siSqpnIvE2i2  active  \
0                                             NaN    True   
19                                            NaN    True   
22                                            NaN    True   
23                                            NaN    True   
26                                            NaN    True   

                         testType.id         testType.name  
0   96baa7ef1443c7a219702eb22e3e68d3  Countermovement Jump  
19  5ee72ddc237b08860c6ac72f52c275f0              Weigh In  
22  0f0017f87fb97445c95b9f2d1133b56b          Drop Landing  
23  0f0017f87fb97445c95b9f2d1133b56b          Drop Landing  
26  7b22e645bdf341c90cf0f5459c957e6a             Drop Jump